In [1]:
from huggingface_hub import notebook_login

notebook_login()



In [4]:
!pip install transformers datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 22.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.0 MB/s eta 0:00:00


In [14]:
import json

# Load the dataset from the JSON file
def load_json_dataset(file_path):
    with open(file_path, 'r') as file:
        dataset = json.load(file)
    return dataset

# Load the dataset from the uploaded file path
dataset_path = "/content/dataset.json"
dataset = load_json_dataset(dataset_path)

# Now you have your dataset loaded and stored in the variable 'dataset'



In [6]:
!pip install pandas


In [16]:
import json
from datasets import Dataset

# Load the JSON file into a list of dictionaries
def load_json_file(file_path):
    with open(file_path, 'r') as file:
        data = json.load(file)
    return data

# Assuming the JSON file is located at '/content/Fine_tune_dataset_29_examples.json'
file_path = '/content/dataset.json'
data = load_json_file(file_path)

# Convert the list of dictionaries to a dictionary-like structure
# where keys are column names and values are lists of data points
# Here, we assume each dictionary in `data` represents a row in the dataset
# and keys in the dictionary represent column names
data_dict = {key: [d[key] for d in data] for key in data[0]}

# Convert the dictionary-like structure to a Hugging Face Dataset
dataset = Dataset.from_dict(data_dict)

# Now you have your dataset loaded and stored in the variable 'dataset'




In [18]:
from transformers import AutoTokenizer

model_checkpoint = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

# Set tokenizer's padding token
tokenizer.pad_token = tokenizer.eos_token

def tokenize_function(examples):
    # Tokenize the inputs and labels
    model_inputs = tokenizer(examples["input"], padding="max_length", truncation=True, max_length=128)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["response"], padding="max_length", truncation=True, max_length=128)["input_ids"]
    model_inputs["labels"] = labels
    return model_inputs

tokenized_datasets = dataset.map(tokenize_function, batched=True)


Map:   0%|          | 0/115 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3892: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [10]:
!pip install accelerate -U
!pip uninstall -y transformers
!pip install transformers[torch]



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 33.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 46.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 47.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━

In [19]:
from transformers import AutoModelForCausalLM, TrainingArguments, Trainer

# Define your model checkpoint
model_checkpoint = "gpt2"  # Example checkpoint name, replace with your desired model

# Define your tokenized dataset
#tokenized_datasets = ...  # You need to replace ... with your tokenized datasets

# Load the model
model = AutoModelForCausalLM.from_pretrained(model_checkpoint)

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="no",
    learning_rate=5e-5,
    weight_decay=0.01,
    per_device_train_batch_size=4,
    num_train_epochs=3,
    logging_dir='./logs',
)


In [20]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets,
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Step,Training Loss


TrainOutput(global_step=87, training_loss=6.352183374865302, metrics={'train_runtime': 742.1905, 'train_samples_per_second': 0.465, 'train_steps_per_second': 0.117, 'total_flos': 22536437760000.0, 'train_loss': 6.352183374865302, 'epoch': 3.0})

In [21]:
import os

# Define the directory path
directory = "./fine_tuned_model"

# Create the directory if it doesn't exist
if not os.path.exists(directory):
    os.makedirs(directory)
trainer.save_model("./fine_tuned_model")



In [22]:
from transformers import pipeline

# Load the fine-tuned model
generator = pipeline("text-generation", model="/content/fine_tuned_model", tokenizer=tokenizer)

# Example prompt
prompt = "Elegant living room with natural light"

# Generate text
generated_texts = generator(prompt, max_length=100, num_return_sequences=1)

for generated_text in generated_texts:
    print(generated_text["generated_text"])


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Elegant living room with natural light windows for outdoor hanging light reflecting light windows the light clean the walls to add, clean and color lines or with, by using to maintain touch of room and clean for colors for use or a and a colors enhance with windows, wall color a light use the light and clean simple to maintain, touch and add a sleek sleek with small like can light the space with, add or a clean and, adding or like clean clean floor and, to create room- touch


In [23]:
prompt = "bedroom with light blue shade"
text =  generator(prompt, max_length=100, num_return_sequences=1)

for generated_text in text:
    print(generated_text["generated_text"])


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


bedroom with light blue shade and a light, color of a and soft shade to add a light light in color or light the lighting color. light a as a to color with light light with light light light, light a touch to add of light a add lighting color, light color light use a and light light lighting, color color create a small with color in light colors add to light. light and light.


In [36]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from sacrebleu.metrics import BLEU

# Function to calculate perplexity
def calculate_perplexity(model, tokenizer, text):
    encodings = tokenizer(text, return_tensors='pt')
    max_length = model.config.n_positions
    stride = 512

    nlls = []
    for i in range(0, encodings.input_ids.size(1), stride):
        begin_loc = max(i + stride - max_length, 0)
        end_loc = min(i + stride, encodings.input_ids.size(1))
        trg_len = end_loc - i    # may be different from stride on last loop
        input_ids = encodings.input_ids[:,begin_loc:end_loc].to(model.device)
        target_ids = input_ids.clone()
        target_ids[:,:-trg_len] = -100

        with torch.no_grad():
            outputs = model(input_ids, labels=target_ids)
            log_likelihood = outputs[0] * trg_len

        nlls.append(log_likelihood)

    ppl = torch.exp(torch.stack(nlls).sum() / end_loc)
    return ppl.item()

# Initialize tokenizers and models for before and after fine-tuning
tokenizer_gpt2 = AutoTokenizer.from_pretrained("gpt2")
model_gpt2 = AutoModelForCausalLM.from_pretrained("gpt2")

tokenizer_fine_tuned = AutoTokenizer.from_pretrained("/content/fine_tuned_model")
model_fine_tuned = AutoModelForCausalLM.from_pretrained("/content/fine_tuned_model")

# Initialize BLEU metric
bleu = BLEU()

# User prompt
user_prompt = "Your user prompt here"  # Replace with actual user prompt
reference_texts = ["Your reference text here"]  # Replace with actual reference text

# Generate response from GPT-2 model
response_gpt2 = pipeline("text-generation", model=model_gpt2, tokenizer=tokenizer_gpt2)(
    user_prompt, max_length=100, truncation=True, num_return_sequences=1
)[0]["generated_text"]

# Generate response from fine-tuned model
response_fine_tuned = pipeline("text-generation", model=model_fine_tuned, tokenizer=tokenizer_fine_tuned)(
    user_prompt, max_length=100, truncation=True, num_return_sequences=1
)[0]["generated_text"]

# Calculate BLEU scores
# BLEU scores should be calculated with the reference being a list of possible translations (even if there's only one)
bleu_score_gpt2 = bleu.corpus_score([response_gpt2], [reference_texts]).score
bleu_score_fine_tuned = bleu.corpus_score([response_fine_tuned], [reference_texts]).score


# Calculate perplexity
perplexity_gpt2 = calculate_perplexity(model_gpt2, tokenizer_gpt2, response_gpt2)
perplexity_fine_tuned = calculate_perplexity(model_fine_tuned, tokenizer_fine_tuned, response_fine_tuned)

# Print results
print(f"Response GPT-2: {response_gpt2}")
print(f"Response Fine-Tuned: {response_fine_tuned}")
print(f"BLEU Score GPT-2: {bleu_score_gpt2}")
print(f"BLEU Score Fine-Tuned: {bleu_score_fine_tuned}")
print(f"Perplexity GPT-2: {perplexity_gpt2}")
print(f"Perplexity Fine-Tuned: {perplexity_fine_tuned}")


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Response GPT-2: Your user prompt here in Microsoft Visual Studio. This app will be converted to a simple web interface for your customers to take advantage of.

The first thing you can do: Go to the "Create Page" menu on the ribbon on your desktop or on the browser. Navigate to the "Create Page" menu where you'll find "Product Pages" and then, under "Page Type" button, double click "Create Page". When the page opens, there will be a window in the
Response Fine-Tuned: Your user prompt here for the classic feel of classic desk. with a bold and sleek design with elegance with a and with a large or slim with of sleek lines a wall or by a or a table a or add a room- like a desk wall add a and while clean or clutter clutter clutter space with with the touch. and a and can or display is and. or and. with Use to enhance and clutter a wall or a to and add clutter clutter add as with in as and enhance and
BLEU Score GPT-2: 0.44029007819264104
BLEU Score Fine-Tuned: 0.4312572865610176
Perplexity 